In [1]:
# Import the necessary libraries and modules
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import re
import csv
import datetime
import time
import pandas as pd

# Define a function for scraping the PM10 values from a website
def PM10_Value_Web_Scraper():
    # Set up Chrome options to run in headless mode to avoid opening a browser window
    options = Options()
    options.headless = True

    # Set up the ChromeDriver service to run the Chrome browser
    service = Service(r'C:\Users\Konstantinos\Desktop\PM10ValueWebScraping\chromedriver_win32')
    
    # Initialize the Chrome browser with the specified service and options
    driver = webdriver.Chrome(service=service, options=options)

    # Navigate to the specified webpage
    url = 'https://www.infoaria.regione.lombardia.it/infoaria/#/stato-attivazione'
    driver.get(url)

    # Get the current date and format it as a string
    today = datetime.date.today()
    yesterday = today - datetime.timedelta(days=1)
    date_string = yesterday.strftime('%Y-%m-%d')

    # Create a list to store the PM10 values for each province
    pm10_values = []

    # Loop through the provinces and extract the PM10 value for each one
    for province in range(1, 12):
        # Construct the XPath string to locate the PM10 value for the current province on the webpage
        xpath_string = "//*[@id='content']/app-stato-attivazione/div[2]/div/div[2]/div[" + str(province) + "]/div/div[4]/div[1]/strong"
        try:
            # Find the HTML element that contains the PM10 value for the current province
            strong_element = driver.find_element(By.XPATH, xpath_string)
            try:
                # Extract the PM10 value from the HTML element as a string
                extracted_pm10_value = strong_element.get_attribute('innerHTML')
                # Define a regular expression pattern to match the PM10 value
                pattern = r'\d+\.\d+'
                # Find all matches of the pattern in the extracted PM10 value
                matches = re.findall(pattern, extracted_pm10_value)
                if matches:
                    # Convert the matched string to a decimal number
                    final_pm10_value = float(matches[0])
                    # Add the PM10 value for the current province to the list of PM10 values
                    pm10_values.append(final_pm10_value)
                else:
                    # Add None to the list of PM10 values if no match is found
                    pm10_values.append(None)
            except:
                # Add None to the list of PM10 values if an exception occurs
                pm10_values.append(None)
        except NoSuchElementException:
            # Add None to the list of PM10 values if the HTML element for the current province cannot be found
            pm10_values.append(None)

    # Quit the browser driver to free up resources
    driver.quit()

    # Create a header row for the CSV file
    header = ['Date', 'Bergamo', 'Brescia', 'Como', 'Cremona', 'Lecco', 'Praise', 'Mantua', 'Milano', 'Monza Brianza', 'Pavia', 'Varese']

    # Create a data row for the CSV file
    data = [date_string] + pm10_values

    # Open the CSV file where the scraped values will be stored
    with open('PM10ValueWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        if f.tell() == 0:
            # Write the header row to the CSV file, if it doesn't exist yet
            writer.writerow(header)
        # Write the data row to the CSV file
        writer.writerow(data)

In [2]:
# Call the PM10_Value_Web_Scraper function to scrape the website and store the data in a CSV file
PM10_Value_Web_Scraper()

# Read the data from the CSV file into a pandas DataFrame
df = pd.read_csv(r'C:\Users\Konstantinos\Desktop\PM10ValueWebScraping\PM10ValueWebScraperDataset.csv')
df

C:\Users\KONSTA~1\AppData\Local\Temp/ipykernel_14352/2649519288.py:17: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
Incompatible release of chromedriver (version 112.0.5615.49) detected in PATH: C:\Users\Konstantinos\Desktop\PM10ValueWebScraping\chromedriver.exe


,Date,Bergamo,Brescia,Como,Cremona,Lecco,Praise,Mantua,Milano,Monza Brianza,Pavia,Varese
0,2023-05-11,NaN,NaN,12.7,9.8,13.0,8.7,NaN,NaN,NaN,10.7,9.8
1,2023-05-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-05-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-05-14,NaN,NaN,NaN,7.0,NaN,7.4,NaN,NaN,NaN,NaN,5.3
